Original code is written by [@Atsuhiko](https://github.com/Atsuhiko)

In [1]:
!nvidia-smi

Tue Nov 12 20:24:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0 Off |                  Off |
|  0%   34C    P8             15W /  450W |     905MiB /  24564MiB |     12%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 基本パラメータ
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
peft_name = "NEFTune_Qwen2.5-0.5B-inst_T4_1ep"
output_dir = "output_neftune"

In [3]:
# ライブラリーのインストール
import torch
from torch import cuda, bfloat16
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/mnt/c/Users/hiroga/Documents/GitHub/til/computer-science/machine-learning/_src/ii-atsuhiko-hands-on/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# モデルの設定
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    # token=token, # HuggingFaceにログインしておけば不要
    quantization_config=bnb_config,  # 量子化
    device_map="auto",
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2" # T4では使えないのでコメントアウト
)

# tokenizerの設定
tokenizer = AutoTokenizer.from_pretrained(
    model_id, padding_side="right", add_eos_token=True
)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id